In [8]:
import os
# import json
import time
# import urllib.error
# import urllib.parse
# import urllib.request
import sys
from graph import *
import csv

from lib import Google_DistMatrix

In [9]:
# ============== Helper func for displaying time
intervals = (
    ('weeks', 604800),  # 60 * 60 * 24 * 7
    ('days', 86400),    # 60 * 60 * 24
    ('hours', 3600),    # 60 * 60
    ('minutes', 60),
    ('seconds', 1),
    )

def display_time(seconds, granularity=2):
    result = []

    for name, count in intervals:
        value = seconds // count
        if value:
            seconds -= value * count
            if value == 1:
                name = name.rstrip('s')
            result.append("{} {}".format(value, name))
    return ', '.join(result[:granularity])
# ===========================================

In [10]:
# Get API key
API_KEY = os.environ.get("DISTANCE_MATRIX_API_KEY",'')
DIST_MATRIX_BASE_URL = "https://maps.googleapis.com/maps/api/distancematrix/json"

DM = Google_DistMatrix(API_KEY)


# key mappings
i_name = 0
i_zid = 1
i_email = 2
i_phone = 3
i_order_beige = 4
i_order_rose = 5
i_order_shirt = 6
i_order_champ = 7
i_address = 8
i_suburb = 9
db = {}

TypeError: 'module' object is not callable

In [4]:
def main():
    # check for correct cmd line arguments ("python3 hamiltonian.py csv_file")
    if (len(sys.argv) < 2 or len(sys.argv) > 3):
        print("Format: python3 hamiltonian.py <csv_file>")
        exit(1)
        
    filename = sys.argv[1]
    db_name = "addresses/example_db.csv"
    if (len(sys.argv) == 3):
        db_name = sys.argv[2]
    
    
    
    # read file and get addresses
    addresses_str,addresses_list = get_addresses_str(filename)
    
    print(addresses_str)
        
    find_path_return = find_path(addresses_str)
    print(find_path_return)

    if find_path_return['status'] != 'OK':
        print("Error: ", find_path_return['status'])
        exit(1)
        
    with open(db_name) as tsv:
        for line in csv.reader(tsv, dialect="excel-tab"):
        
            f_name = filename[:-4] # remove ".csv"
            suburb = f_name.split("/")[1].split("_")[0]
        
            # store only if suburb matches file called
            if line[i_suburb] != suburb:
                print(suburb)
                print(line[i_suburb])
                continue
                
            row = {}
            row["name"] = line[i_name]
            row["zid"] = line[i_zid]
            row["email"] = line[i_email]
            row["phone"] = line[i_phone]
            row["order_beige"] = line[i_order_beige]
            row["order_rose"] = line[i_order_rose]
            row["order_shirt"] = line[i_order_shirt]
            row["order_champ"] = line[i_order_champ]
            row["human_address"] = line[i_address]
            row["suburb"] = line[i_suburb]
            
            # get index of human_input
            addr_index = 0
            # print("searching for ", line[i_address])
            for a in addresses_list:
                if a == line[i_address]:
                    break
                addr_index += 1
                
            # if address not found, don't store
            if addr_index == len(addresses_list):
                # print("address not found")
                # print("addresses_list = ", addresses_list)
                # print("line[i_address] = ", line[i_address])
                continue
            # print(addr_index)
            # print(addresses_list)
            # match that with index of API response
            # print(find_path_return['destination_addresses'])
            row["API_address"] = find_path_return['destination_addresses'][addr_index]
            
            db[find_path_return['destination_addresses'][addr_index]] = row
            
            # print(row)
            
   
    # print("======================= DB ======================")
    # print(db)
    # print("======================= ========== ======================")

    
    # ===== Creating graph G1 =====
    G1 = Graph(find_path_return["destination_addresses"]) 
    
    # ===== UNCOMMENT DEPENDING ON DISTANCE OR TIME =====
    # G1.graph = matrix_by_distance(find_path_return)
    G1.graph = matrix_by_time(find_path_return)
    # ===================================================


    path,trip_time = G1.solve()
    print_final_route(path,find_path_return["destination_addresses"],trip_time)

    print("Laters.")